In [1]:
import xml.etree.ElementTree as ET
import glob
import os
import cv2
import numpy as np
from shutil import copyfile

if __name__ == '__main__':

    print("[GAN] Create label data....")

    inputFolder = r"C:\img\gan_mura\Split_data"
    outputFolder = r"C:\img\gan_mura\Split_data_pin_out"

    xmlList = []
    selectedDefect = []

    ### Drawimage parameter ###
    objectList = []
    thickness = 1
    color = (255,255,255)

    ### Initialize ###
    if os.path.isdir(outputFolder) == False:
        os.mkdir(outputFolder)

    if os.path.isdir(inputFolder) == False:
        print("Cannot find input folder." + "(" + inputFolder + ")")
        pass

    ### Get XML list ###
    xmlList = glob.glob(inputFolder + "\\" + "*.xml")

    ### Load XML ###

    for xmlpath in xmlList:

        objectList.clear()

        filename_w_ext = os.path.basename(xmlpath) # file name with extension
        filename, file_extension = os.path.splitext(filename_w_ext)
        imageName = filename + "_GAN" + ".jpg"
        originalImagePath = inputFolder + "\\" + filename + ".jpg"

        ### 從檔案載入並解析 XML 資料 ###
        tree = ET.parse(xmlpath)
        root = tree.getroot()

        ### Get image size (w * h) ###
        width = int(root.find("size").find("width").text)
        height = int(root.find("size").find("height").text)
        print(width, height)

        ### Initial default image ###
        newImage = np.zeros((height, width, 3), np.uint8)
        newImage.fill(0)

        for ob in root.findall("object"):
            xmin = int(ob.find('bndbox').find("xmin").text)
            ymin = int(ob.find('bndbox').find("ymin").text)
            xmax = int(ob.find('bndbox').find("xmax").text)
            ymax = int(ob.find('bndbox').find("ymax").text)
            label_name = str(ob.find('name').text)

            objectUnit = {'name': label_name, 'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax}


            print(objectUnit)

            ### Draw Image ###
            # 微調座標 (邊界值內縮，以畫線)
            dxmin, dymin, dxmax, dymax = objectUnit["xmin"],objectUnit["ymin"],objectUnit["xmax"],objectUnit["ymax"]
            if dxmin == 0:
                dxmin += 1
            if dymin == 0:
                dymin += 1
            if dxmax == width:
                dxmax -= 1
            if dymax == height:
                dymax -= 1

            # 直線
            # cv2.line(影像, 開始座標, 結束座標, 顏色, 線條寬度)
            # 方框
            # cv2.rectangle(影像, 頂點座標, 對向頂點座標, 顏色, 線條寬度)
            # 橢圓形
            # cv2.ellipse(影像, 中心座標, 軸長, 旋轉角度, 起始角度, 結束角度, 顏色, 線條寬度)

#             if objectUnit["name"] == "Slit_Mura_H":
#                 y = int((dymin + dymax)/2)
#                 cv2.line(newImage, (dxmin,y),(dxmax,y),color,thickness)
#                 objectList.append(objectUnit)

            if objectUnit["name"] == "Pin_Mura":
                x = int((dxmin + dxmax) / 2)
                y = int((dymin + dymax) / 2)
                x_length = int(abs(dxmin - dxmax) / 2)
                y_length = int(abs(dymin - dymax) / 2)
                cv2.ellipse(newImage, (x,y), (x_length,y_length),0,0,360,color,thickness)
                objectList.append(objectUnit)

#             if objectUnit["name"] == "Coin_Mura":
#                 x = int((dxmin + dxmax) / 2)
#                 y = int((dymin + dymax) / 2)
#                 x_length = int(abs(dxmin - dxmax) / 2)
#                 y_length = int(abs(dymin - dymax) / 2)
#                 cv2.ellipse(newImage, (x,y), (x_length,y_length),0,0,360,color,thickness)
#                 objectList.append(objectUnit)

#             if objectUnit["name"] == "Tokki":
#                 x = int((dxmin + dxmax) / 2)
#                 y = int((dymin + dymax) / 2)
#                 x_length = int(abs(dxmin - dxmax) / 2)
#                 y_length = int(abs(dymin - dymax) / 2)
#                 cv2.ellipse(newImage, (x,y), (x_length,y_length),0,0,360,color,thickness)
#                 objectList.append(objectUnit)

#             if objectUnit["name"] == "break_TK":
#                 pass

#             if objectUnit["name"] == "Other":
#                 continue

#                 x = int((dxmin + dxmin) / 2)
#                 y = int((dymin + dymax) / 2)
#                 x_length = int(abs(dxmin - dxmax) / 2)
#                 y_length = int(abs(dymin - dymax) / 2)
#                 cv2.ellipse(newImage, (x,y), (x_length,y_length),0,0,360,color,thickness)
#                 objectList.append(objectUnit)

        ### Show image ###
        # cv2.imshow(filename, newImage)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()


        if len(objectList) != 0:

            ### Save label image ###
            outputPath = outputFolder + "\\" + imageName
            cv2.imwrite(outputPath, newImage, [cv2.IMWRITE_JPEG_QUALITY, 100])

            ### Clone original image ###
            if os.path.isfile(originalImagePath) == True:
                copyfile(originalImagePath, originalImagePath.replace(inputFolder, outputFolder))



            ### Check image ###
            originalImage = cv2.imread(originalImagePath)
#             gray = originalImage.copy()
#             xOffset = 10
#             yOffset = 10

#             roi = gray[yOffset:gray.shape[0]-yOffset, xOffset:gray.shape[1] - xOffset] #roi = gray[y1:y2, x1:x2]
#             eqImage = cv2.equalizeHist(roi)
#             checkImage2 = originalImage[yOffset:yOffset+eqImage.shape[0], xOffset:xOffset + eqImage.shape[1]]

            for objectUnit in objectList:
                cv2.rectangle(originalImage, (objectUnit["xmin"],objectUnit["ymin"]), (objectUnit["xmax"],objectUnit["ymax"]), color, -1)

#             checkImage = cv2.hconcat([originalImage, checkImage2])
            checkImage = cv2.hconcat([originalImage, newImage])

            ### Show image ###
            #cv2.imshow("Check image", checkImage)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()

    print("Done...")


[GAN] Create label data....
512 512
{'name': 'Slit_Mura_H', 'xmin': 17, 'ymin': 217, 'xmax': 512, 'ymax': 246}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 135, 'xmax': 496, 'ymax': 164}
512 512
{'name': 'Slit_Mura_H', 'xmin': 12, 'ymin': 251, 'xmax': 512, 'ymax': 277}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 251, 'xmax': 495, 'ymax': 277}
512 512
{'name': 'Slit_Mura_H', 'xmin': 12, 'ymin': 177, 'xmax': 512, 'ymax': 203}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 177, 'xmax': 495, 'ymax': 203}
512 512
{'name': 'Slit_Mura_H', 'xmin': 16, 'ymin': 122, 'xmax': 512, 'ymax': 151}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 122, 'xmax': 490, 'ymax': 151}
512 512
{'name': 'Slit_Mura_H', 'xmin': 16, 'ymin': 46, 'xmax': 512, 'ymax': 75}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 46, 'xmax': 490, 'ymax': 75}
512 512
{'name': 'Slit_Mura_H', 'xmin': 12, 'ymin': 150, 'xmax': 512, 'ymax': 181}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 150, 'xmax': 496,

{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 271, 'xmax': 208, 'ymax': 300}
{'name': 'Other', 'xmin': 209, 'ymin': 274, 'xmax': 403, 'ymax': 296}
512 512
{'name': 'Slit_Mura_H', 'xmin': 18, 'ymin': 193, 'xmax': 512, 'ymax': 222}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 193, 'xmax': 208, 'ymax': 222}
{'name': 'Other', 'xmin': 209, 'ymin': 196, 'xmax': 403, 'ymax': 218}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 122, 'xmax': 512, 'ymax': 153}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 122, 'xmax': 488, 'ymax': 153}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 47, 'xmax': 512, 'ymax': 78}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 47, 'xmax': 488, 'ymax': 78}
512 512
{'name': 'Slit_Mura_H', 'xmin': 12, 'ymin': 152, 'xmax': 512, 'ymax': 178}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 152, 'xmax': 494, 'ymax': 178}
512 512
{'name': 'Slit_Mura_H', 'xmin': 12, 'ymin': 76, 'xmax': 512, 'ymax': 102}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin

{'name': 'Other', 'xmin': 400, 'ymin': 405, 'xmax': 435, 'ymax': 432}
512 512
{'name': 'Slit_Mura_H', 'xmin': 21, 'ymin': 369, 'xmax': 512, 'ymax': 397}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 369, 'xmax': 412, 'ymax': 397}
{'name': 'Other', 'xmin': 411, 'ymin': 361, 'xmax': 459, 'ymax': 403}
512 512
{'name': 'Slit_Mura_H', 'xmin': 21, 'ymin': 293, 'xmax': 512, 'ymax': 321}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 293, 'xmax': 412, 'ymax': 321}
{'name': 'Other', 'xmin': 411, 'ymin': 285, 'xmax': 459, 'ymax': 327}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 343, 'xmax': 512, 'ymax': 371}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 343, 'xmax': 379, 'ymax': 371}
{'name': 'Other', 'xmin': 382, 'ymin': 341, 'xmax': 433, 'ymax': 377}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 267, 'xmax': 512, 'ymax': 295}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 267, 'xmax': 379, 'ymax': 295}
{'name': 'Other', 'xmin': 382, 'ymin': 265, 'xmax': 433, 'ym

{'name': 'Slit_Mura_H', 'xmin': 13, 'ymin': 326, 'xmax': 512, 'ymax': 354}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 326, 'xmax': 496, 'ymax': 354}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 86, 'xmax': 512, 'ymax': 116}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 86, 'xmax': 497, 'ymax': 116}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 9, 'xmax': 512, 'ymax': 39}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 9, 'xmax': 497, 'ymax': 39}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 121, 'xmax': 512, 'ymax': 149}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 121, 'xmax': 490, 'ymax': 149}
512 512
{'name': 'Slit_Mura_H', 'xmin': 14, 'ymin': 43, 'xmax': 512, 'ymax': 71}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 43, 'xmax': 490, 'ymax': 71}
512 512
{'name': 'Slit_Mura_H', 'xmin': 21, 'ymin': 195, 'xmax': 512, 'ymax': 225}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 195, 'xmax': 494, 'ymax': 225}
512 512
{'name': 'Slit_Mura_H'

{'name': 'Pin_Mura', 'xmin': 279, 'ymin': 132, 'xmax': 339, 'ymax': 192}
512 512
{'name': 'Pin_Mura', 'xmin': 279, 'ymin': 52, 'xmax': 339, 'ymax': 112}
{'name': 'Pin_Mura', 'xmin': 110, 'ymin': 471, 'xmax': 168, 'ymax': 499}
512 512
{'name': 'Pin_Mura', 'xmin': 147, 'ymin': 67, 'xmax': 201, 'ymax': 123}
512 512
{'name': 'Pin_Mura', 'xmin': 147, 'ymin': 0, 'xmax': 201, 'ymax': 47}
512 512
{'name': 'Pin_Mura', 'xmin': 325, 'ymin': 481, 'xmax': 361, 'ymax': 502}
512 512
{'name': 'Pin_Mura', 'xmin': 146, 'ymin': 67, 'xmax': 200, 'ymax': 123}
512 512
{'name': 'Pin_Mura', 'xmin': 146, 'ymin': 0, 'xmax': 200, 'ymax': 47}
512 512
{'name': 'Tokki', 'xmin': 400, 'ymin': 181, 'xmax': 432, 'ymax': 212}
512 512
{'name': 'Tokki', 'xmin': 400, 'ymin': 99, 'xmax': 432, 'ymax': 130}
512 512
{'name': 'Slit_Mura_H', 'xmin': 155, 'ymin': 182, 'xmax': 512, 'ymax': 213}
512 512
{'name': 'Slit_Mura_H', 'xmin': 0, 'ymin': 182, 'xmax': 491, 'ymax': 213}
512 512
{'name': 'Slit_Mura_H', 'xmin': 155, 'ymin': 109

...

{'name': 'Pin_Mura', 'xmin': 382, 'ymin': 158, 'xmax': 446, 'ymax': 224}
512 512
{'name': 'Pin_Mura', 'xmin': 180, 'ymin': 317, 'xmax': 237, 'ymax': 370}
512 512
{'name': 'Pin_Mura', 'xmin': 180, 'ymin': 233, 'xmax': 237, 'ymax': 286}
512 512
{'name': 'Pin_Mura', 'xmin': 70, 'ymin': 425, 'xmax': 129, 'ymax': 472}
512 512
{'name': 'Pin_Mura', 'xmin': 70, 'ymin': 341, 'xmax': 129, 'ymax': 388}
512 512
{'name': 'Pin_Mura', 'xmin': 369, 'ymin': 208, 'xmax': 460, 'ymax': 279}
512 512
{'name': 'Pin_Mura', 'xmin': 369, 'ymin': 142, 'xmax': 460, 'ymax': 213}
512 512
{'name': 'Pin_Mura', 'xmin': 184, 'ymin': 293, 'xmax': 252, 'ymax': 357}
512 512
{'name': 'Pin_Mura', 'xmin': 184, 'ymin': 208, 'xmax': 252, 'ymax': 272}
512 512
{'name': 'Pin_Mura', 'xmin': 263, 'ymin': 371, 'xmax': 314, 'ymax': 414}
512 512
{'name': 'Pin_Mura', 'xmin': 263, 'ymin': 286, 'xmax': 314, 'ymax': 329}
512 512
{'name': 'Pin_Mura', 'xmin': 381, 'ymin': 228, 'xmax': 456, 'ymax': 297}
512 512
{'name': 'Pin_Mura', 'xmin': 3

512 512
{'name': 'Pin_Mura', 'xmin': 250, 'ymin': 404, 'xmax': 317, 'ymax': 458}
512 512
{'name': 'Pin_Mura', 'xmin': 250, 'ymin': 321, 'xmax': 317, 'ymax': 375}
512 512
{'name': 'Coin_Mura', 'xmin': 120, 'ymin': 354, 'xmax': 187, 'ymax': 416}
512 512
{'name': 'Coin_Mura', 'xmin': 120, 'ymin': 275, 'xmax': 187, 'ymax': 337}
512 512
{'name': 'Pin_Mura', 'xmin': 71, 'ymin': 267, 'xmax': 132, 'ymax': 314}
512 512
{'name': 'Pin_Mura', 'xmin': 71, 'ymin': 183, 'xmax': 132, 'ymax': 230}
512 512
{'name': 'Pin_Mura', 'xmin': 200, 'ymin': 14, 'xmax': 271, 'ymax': 49}
512 512
{'name': 'Pin_Mura', 'xmin': 199, 'ymin': 434, 'xmax': 258, 'ymax': 488}
512 512
{'name': 'Pin_Mura', 'xmin': 199, 'ymin': 350, 'xmax': 258, 'ymax': 404}
512 512
{'name': 'Coin_Mura', 'xmin': 338, 'ymin': 465, 'xmax': 385, 'ymax': 508}
{'name': 'Coin_Mura', 'xmin': 399, 'ymin': 382, 'xmax': 446, 'ymax': 430}
512 512
{'name': 'Coin_Mura', 'xmin': 432, 'ymin': 451, 'xmax': 485, 'ymax': 497}
{'name': 'Coin_Mura', 'xmin': 338, 

{'name': 'Pin_Mura', 'xmin': 211, 'ymin': 438, 'xmax': 267, 'ymax': 495}
512 512
{'name': 'Pin_Mura', 'xmin': 211, 'ymin': 354, 'xmax': 267, 'ymax': 411}
512 512
{'name': 'Coin_Mura', 'xmin': 403, 'ymin': 378, 'xmax': 450, 'ymax': 433}
{'name': 'Coin_Mura', 'xmin': 333, 'ymin': 469, 'xmax': 386, 'ymax': 512}
512 512
{'name': 'Coin_Mura', 'xmin': 428, 'ymin': 439, 'xmax': 475, 'ymax': 488}
{'name': 'Coin_Mura', 'xmin': 403, 'ymin': 299, 'xmax': 450, 'ymax': 354}
{'name': 'Coin_Mura', 'xmin': 333, 'ymin': 390, 'xmax': 386, 'ymax': 439}
512 512
{'name': 'Pin_Mura', 'xmin': 267, 'ymin': 451, 'xmax': 331, 'ymax': 512}
512 512
{'name': 'Pin_Mura', 'xmin': 267, 'ymin': 367, 'xmax': 331, 'ymax': 428}
512 512
{'name': 'Coin_Mura', 'xmin': 241, 'ymin': 63, 'xmax': 299, 'ymax': 117}
{'name': 'Coin_Mura', 'xmin': 122, 'ymin': 349, 'xmax': 177, 'ymax': 397}
{'name': 'Coin_Mura', 'xmin': 319, 'ymin': 356, 'xmax': 368, 'ymax': 397}
{'name': 'Coin_Mura', 'xmin': 251, 'ymin': 463, 'xmax': 320, 'ymax': 

{'name': 'Coin_Mura', 'xmin': 68, 'ymin': 393, 'xmax': 111, 'ymax': 438}
{'name': 'Coin_Mura', 'xmin': 317, 'ymin': 291, 'xmax': 373, 'ymax': 343}
{'name': 'Coin_Mura', 'xmin': 71, 'ymin': 159, 'xmax': 114, 'ymax': 201}
{'name': 'Coin_Mura', 'xmin': 74, 'ymin': 108, 'xmax': 119, 'ymax': 153}
512 512
{'name': 'Coin_Mura', 'xmin': 158, 'ymin': 132, 'xmax': 238, 'ymax': 209}
{'name': 'Coin_Mura', 'xmin': 63, 'ymin': 74, 'xmax': 109, 'ymax': 123}
512 512
{'name': 'Coin_Mura', 'xmin': 68, 'ymin': 312, 'xmax': 111, 'ymax': 357}
{'name': 'Coin_Mura', 'xmin': 317, 'ymin': 210, 'xmax': 373, 'ymax': 262}
{'name': 'Coin_Mura', 'xmin': 71, 'ymin': 78, 'xmax': 114, 'ymax': 120}
{'name': 'Coin_Mura', 'xmin': 74, 'ymin': 27, 'xmax': 119, 'ymax': 72}
512 512
{'name': 'Coin_Mura', 'xmin': 158, 'ymin': 51, 'xmax': 238, 'ymax': 128}
{'name': 'Coin_Mura', 'xmin': 63, 'ymin': 0, 'xmax': 109, 'ymax': 42}
512 512
{'name': 'Coin_Mura', 'xmin': 473, 'ymin': 19, 'xmax': 512, 'ymax': 72}
{'name': 'Coin_Mura', 'x

512 512
{'name': 'Pin_Mura', 'xmin': 251, 'ymin': 280, 'xmax': 303, 'ymax': 329}
512 512
{'name': 'Pin_Mura', 'xmin': 84, 'ymin': 366, 'xmax': 138, 'ymax': 417}
512 512
{'name': 'Pin_Mura', 'xmin': 84, 'ymin': 281, 'xmax': 138, 'ymax': 332}
512 512
{'name': 'Coin_Mura', 'xmin': 476, 'ymin': 23, 'xmax': 512, 'ymax': 77}
{'name': 'Coin_Mura', 'xmin': 472, 'ymin': 225, 'xmax': 512, 'ymax': 282}
{'name': 'Coin_Mura', 'xmin': 333, 'ymin': 20, 'xmax': 383, 'ymax': 74}
512 512
{'name': 'Coin_Mura', 'xmin': 9, 'ymin': 23, 'xmax': 56, 'ymax': 77}
{'name': 'Coin_Mura', 'xmin': 98, 'ymin': 348, 'xmax': 146, 'ymax': 401}
{'name': 'Coin_Mura', 'xmin': 5, 'ymin': 225, 'xmax': 52, 'ymax': 282}
512 512
{'name': 'Coin_Mura', 'xmin': 472, 'ymin': 147, 'xmax': 512, 'ymax': 204}
512 512
{'name': 'Coin_Mura', 'xmin': 98, 'ymin': 270, 'xmax': 146, 'ymax': 323}
{'name': 'Coin_Mura', 'xmin': 5, 'ymin': 147, 'xmax': 52, 'ymax': 204}
512 512
{'name': 'Coin_Mura', 'xmin': 66, 'ymin': 388, 'xmax': 118, 'ymax': 44